<a href="https://colab.research.google.com/github/MatDawit/AI4ALL/blob/jackie-ver/cleanedstartupdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Purpose of Cleaning This Dataset**

This dataset will be used in a random forest algorithm machine learning model to predict whether or not a startup is still operating.

In [ ]:
# importing key libraries for this project
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# importing the dataset
from google.colab import files
uploaded = files.upload()

df = pd.read_csv('big_startup_secsees_dataset.csv')

df.head()

Saving big_startup_secsees_dataset.csv to big_startup_secsees_dataset (6).csv


,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at
0,/organization/-fame,#fame,http://livfame.com,Media,10000000,operating,IND,16,Mumbai,Mumbai,1,NaN,2015-01-05,2015-01-05
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,700000,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014-03-01,2014-10-14
2,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,3406878,operating,NaN,NaN,NaN,NaN,1,NaN,2014-01-30,2014-01-30
3,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,2000000,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008-03-19,2008-03-19
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014-07-24,2014-07-24


In [ ]:
# viewing the size of the dataset before cleaning
print("Number of rows in the dataset:", len(df))

Number of rows in the dataset: 66368


In [ ]:
# exploring the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66368 entries, 0 to 66367
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   permalink          66368 non-null  object
 1   name               66367 non-null  object
 2   homepage_url       61310 non-null  object
 3   category_list      63220 non-null  object
 4   funding_total_usd  66368 non-null  object
 5   status             66368 non-null  object
 6   country_code       59410 non-null  object
 7   state_code         57821 non-null  object
 8   region             58338 non-null  object
 9   city               58340 non-null  object
 10  funding_rounds     66368 non-null  int64 
 11  founded_at         51147 non-null  object
 12  first_funding_at   66344 non-null  object
 13  last_funding_at    66368 non-null  object
dtypes: int64(1), object(13)
memory usage: 7.1+ MB


**Data Cleaning Process**


*   `Removing duplicates`: removing duplicates ensures data isn't skewed
*   `Removing unneeded columns`: optimizes load times by removing irrelevant columns
*   `Fixing data types`: fixing datatypes for clarity and mathematical uses (see, funding_total_usd)
*   `Separating categories`: by duplicating each row to allow for 1 category per row, this allows for a more expansive analysis to be done with categories
*   `Removing significant outliers`: prevents outliers from skewing the dataset and the ML prediction model
*   `Removing rows with any amount of Nulls/NaN/NaT`: ensures data quality by having full information on every row because all columns remaining are important to the prediction capability of the model






In [ ]:
# dropping any duplicate rows
df.drop_duplicates()

# view amount of rows after dropping duplicates
print("Number of rows in the dataset:", len(df))

Number of rows in the dataset: 66368


No duplicates in the dataset.

In [ ]:
# removing unneeded columns
df = df.drop(['state_code', 'region', 'city', 'permalink', 'name', 'homepage_url'], axis='columns')

In [ ]:
# view remaining columns
df.columns.tolist()

['category_list',
 'funding_total_usd',
 'status',
 'country_code',
 'funding_rounds',
 'founded_at',
 'first_funding_at',
 'last_funding_at']

**Columns Remaining and Why**


*   `Category_list`: there might be a trend of certain types of categories failing more than succeeding
*   `Funding_total_usd`: amount of funding (low or high) could be a reason why startups are failing or succeeding because it takes time to break even
*   `Status`: status is whether or not a startup is closed (0) or operating (1)
*   `Country_code`: certain countries might have a higher % of successful startups than others
*   `Funding_rounds`: the amount of funding rounds correlates to investor interest, more interest should generally mean a higher chance of succeeding
*   `Founded_at`: certain timeframes (2008, 2020 for example) are much different financially than other years
*  ` First_funding_at`: see Founded_at for more details
*   `Last_funding_at`: later funding indicates attempts to scale and expand the startup operations



In [ ]:
# convert dates to datetime
df['founded_at'] = pd.to_datetime(df['founded_at'], errors='coerce')
df['first_funding_at'] = pd.to_datetime(df['first_funding_at'], errors='coerce')
df['last_funding_at'] = pd.to_datetime(df['last_funding_at'], errors='coerce')

In [ ]:
# convert funding total to numeric
df['funding_total_usd'] = pd.to_numeric(df['funding_total_usd'], errors='coerce')

In [ ]:
# before exploding category list, viewing amount of operating or closed startups to gauge whether
# or not duplicating rows will exasperbate the data imbalance

df['status'].value_counts()

,count
status,
operating,53034
closed,6238
acquired,5549
ipo,1547


In [ ]:
# actually exploding category list
df = df.assign(category_list=df['category_list'].str.split('|')).explode('category_list')

In [ ]:
# viewing results
df['status'].value_counts()

,count
status,
operating,126370
closed,13600
acquired,12829
ipo,3007


**Prior to Removing Rows w/ Null Values**

`Before Exploding`: Closed makes up 9.3% of dataset

`After Exploding`: Closed makes up 8.7% of dataset

Negligible impact on dataset from using the explode function.

In [ ]:
# assessing outliers in the dataset
df['last_funding_at'].sort_values(ascending=False).head(10)

,last_funding_at
63921,2115-12-01
63921,2115-12-01
63921,2115-12-01
46956,2105-05-01
46956,2105-05-01
46956,2105-05-01
46956,2105-05-01
36653,2015-12-12
36653,2015-12-12
36653,2015-12-12


63921 and 46956 need to be removed: those dates don't exist.

In [ ]:
# removing outliers date time
df = df[df['last_funding_at'].dt.year <= 2026]

In [ ]:
# continuing to assess outliers
df['last_funding_at'].sort_values().head(10)

,last_funding_at
28378,1971-01-01
8786,1973-04-15
3728,1977-05-15
3728,1977-05-15
3728,1977-05-15
3728,1977-05-15
3728,1977-05-15
744,1979-01-01
36251,1980-01-01
6049,1982-03-20


In [ ]:
# continuing to assess outliers
df['first_funding_at'].sort_values(ascending=False).head(10)

,first_funding_at
30484,2015-12-05
56806,2015-12-04
36147,2015-12-04
48845,2015-12-04
56806,2015-12-04
66202,2015-12-04
36894,2015-12-04
32485,2015-12-04
55124,2015-12-03
61114,2015-12-03


In [ ]:
# continuing to assess outliers
df['first_funding_at'].sort_values().head(10)

,first_funding_at
61699,1960-01-01
61699,1960-01-01
28378,1971-01-01
8786,1973-04-15
3728,1977-05-15
3728,1977-05-15
3728,1977-05-15
3728,1977-05-15
3728,1977-05-15
744,1979-01-01


In [ ]:
# removing all rows w/ any null, NaN, or NaT
df = df.dropna()

In [ ]:
# view size of dataset after removing all nulls
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100963 entries, 1 to 66366
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category_list      100963 non-null  object        
 1   funding_total_usd  100963 non-null  float64       
 2   status             100963 non-null  object        
 3   country_code       100963 non-null  object        
 4   funding_rounds     100963 non-null  int64         
 5   founded_at         100963 non-null  datetime64[ns]
 6   first_funding_at   100963 non-null  datetime64[ns]
 7   last_funding_at    100963 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(1), object(3)
memory usage: 6.9+ MB


In [ ]:
# checking outliers of total funding

from scipy.stats import zscore
import numpy as np

# calculate z-scores
z_scores = zscore(df['funding_total_usd'])

# Step 2: Keep only rows where |z| < 3 (i.e., not outliers)
df = df[np.abs(z_scores) < 3]

In [ ]:
# viewing what was removed
print("Number of rows in the dataset:", len(df))

Number of rows in the dataset: 100697


In [ ]:
# viewing amount of each status after data cleaning
df['status'].value_counts()

,count
status,
operating,82696
acquired,9036
closed,6821
ipo,2144


Closed is now 6.7% of dataset.

In [ ]:
# encoding status by 0 (closed) or 1 (acquired, ipo, or operating)
df['status'] = df['status'].apply(lambda x: 0 if x == 'closed' else 1)

In [ ]:
# view results of encoding
df.head()

,category_list,funding_total_usd,status,country_code,funding_rounds,founded_at,first_funding_at,last_funding_at
1,Application Platforms,700000.0,1,USA,2,2014-09-04,2014-03-01,2014-10-14
1,Real Time,700000.0,1,USA,2,2014-09-04,2014-03-01,2014-10-14
1,Social Network Media,700000.0,1,USA,2,2014-09-04,2014-03-01,2014-10-14
3,Curated Web,2000000.0,1,CHN,1,2007-01-01,2008-03-19,2008-03-19
6,Biotechnology,762851.0,1,CAN,2,1997-01-01,2009-09-11,2009-12-21


**Exploratory Data Analysis**

(now that the data is fully cleaned)
*   Countries (top 10) with the most still not closed startups
*   Categories (top 10) with the most not closed startups
*   Categories (top 10) with the most closed startups
*   Amount of unique countries represented in the dataset
*   Average funding of closed and not closed startups
*   Average funding rounds of closed and not closed startups


In [ ]:
# what percent of startups are still operating in each country?
operating_by_country = df[df['status'] == 1].groupby('country_code')['status'].count().sort_values(ascending=False)

print(operating_by_country.head(10))

country_code
USA    59259
GBR     6165
CAN     3107
IND     1985
FRA     1850
ISR     1677
ESP     1557
DEU     1462
CHN     1088
SGP      979
Name: status, dtype: int64


In [ ]:
# what category has the most startups that are still operating?
industry_survival = df[df['status'] == 1].groupby('category_list')['status'].count().sort_values(ascending=False)

print(industry_survival.head(10))

category_list
Software               5711
Mobile                 3633
Biotechnology          2777
E-Commerce             2541
Enterprise Software    1962
Social Media           1804
Curated Web            1749
Advertising            1647
SaaS                   1477
Health Care            1395
Name: status, dtype: int64


In [ ]:
# what category has the most startups that are closed?
industry_failure = df[df['status'] == 0].groupby('category_list')['status'].count().sort_values(ascending=False)

print(industry_failure.head(10))

category_list
Software               390
Curated Web            311
Mobile                 305
Social Media           253
E-Commerce             211
Games                  157
Advertising            152
Biotechnology          124
Internet               105
Enterprise Software     97
Name: status, dtype: int64


In [ ]:
# how many unique countries are there remaining in the dataset?
df['country_code'].nunique()

122

There are 3 countries with no successful startups.

In [ ]:
# average and median funding for closed (0) vs acquired, operating, or ipo (1)
funding_stats = df.groupby('status')['funding_total_usd'].agg(['mean', 'median', 'count'])

print(funding_stats)

                mean     median  count
status                                
0       6.470032e+06   690000.0   6821
1       1.406111e+07  1700000.0  93876


Startups that are closed have received significantly less funding: a possible correlation between funding and success.

In [ ]:
# funding rounds by successful startup or failed startup
funding_rounds = df.groupby('status')['funding_rounds'].agg(['mean', 'median', 'count'])

print(funding_rounds)

            mean  median  count
status                         
0       1.611934     1.0   6821
1       2.133634     2.0  93876


No noticeable difference in funding rounds between closed and still operating.